In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Conv1D, MaxPooling1D, Flatten, Add, ReLU, LSTM, Reshape, Concatenate, Activation
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.regularizers import l1_l2

from sklearn.metrics import classification_report, confusion_matrix

import pickle
import os
from contextlib import redirect_stdout

In [ ]:
#import PTB data
X_ptb_train = pd.read_csv('data/processed/PTB/X_ptb_train.csv')
y_ptb_train = pd.read_csv('data/processed/PTB/y_ptb_train.csv')

X_ptb_train_sm = pd.read_csv('data/processed/PTB/X_ptb_train_sm.csv')
y_ptb_train_sm = pd.read_csv('data/processed/PTB/y_ptb_train_sm.csv')

X_ptb_val = pd.read_csv('data/processed/PTB/X_ptb_val.csv')
y_ptb_val = pd.read_csv('data/processed/PTB/y_ptb_val.csv')

X_ptb_test = pd.read_csv('data/processed/PTB/X_ptb_test.csv')
y_ptb_test = pd.read_csv('data/processed/PTB/y_ptb_test.csv')


display(X_ptb_train.shape)
display(y_ptb_train.shape)

display(X_ptb_train_sm.shape)
display(y_ptb_train_sm.shape)

display(X_ptb_val.shape)
display(y_ptb_val.shape)

display(X_ptb_test.shape)
display(y_ptb_test.shape)



# Reshape the data for 1D CNN
X_ptb_train_cnn = np.expand_dims(X_ptb_train, axis=2)
X_ptb_train_sm_cnn = np.expand_dims(X_ptb_train_sm, axis=2)
X_ptb_val_cnn = np.expand_dims(X_ptb_val, axis=2)
X_ptb_test_cnn = np.expand_dims(X_ptb_test, axis=2)

display(X_ptb_train_cnn.shape)
display(y_ptb_train.shape)

display(X_ptb_train_sm_cnn.shape)
display(y_ptb_train_sm.shape)

display(X_ptb_val_cnn.shape)
display(y_ptb_val.shape)

display(X_ptb_test_cnn.shape)
display(y_ptb_test.shape)

In [ ]:
#Load model for that will be retrained
model_trained = load_model('best_dl_model/cnn8_sm_lrexpdec1e-3_earlystop_bs512_epoch_52_valloss_0.0676.keras')

model_trained.summary()


# Extract features 
feature_extractor = Model(inputs=model_trained.input, outputs=model_trained.get_layer('max_pooling1d_4').output)


feature_extractor.summary()

In [ ]:
#Freeze all convolutional layers in feature_extractor
for layer in feature_extractor.layers:
    layer.trainable = False

In [ ]:
#tested transfer models

#transfer 2, with added dropout
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.1)(x)
output_layer = Dense(2, activation='softmax')(x)


#transfer 3, with changed dropout
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.4)(x)
output_layer = Dense(2, activation='softmax')(x)


#transfer 4, with dropout and batch normalization 
x = Flatten()(x)
x = Dense(32)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(32)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.3)(x)

output_layer = Dense(2, activation='softmax')(x)

transfer_model = Model(inputs=input_layer, outputs=output_layer)


#transfer 5, with changed dropout and batch normalization 
x = Flatten()(x)
x = Dense(32)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.4)(x)

x = Dense(32)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.4)(x)

output_layer = Dense(2, activation='softmax')(x)

transfer_model = Model(inputs=input_layer, outputs=output_layer)


#transfer 6
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.3)(x)
output_layer = Dense(2, activation='softmax')(x)


#transfer 7 
x = Flatten()(x)
x = Dense(32)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.1)(x)

x = Dense(32)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.1)(x)

output_layer = Dense(2, activation='softmax')(x)

transfer_model = Model(inputs=input_layer, outputs=output_layer)


#transfer 8
x = Flatten()(x)
x = Dense(32)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)

x = Dense(32)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)

output_layer = Dense(2, activation='softmax')(x)

transfer_model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# Build new classifier for 2 class problem

# Input same shape as original
input_layer = Input(shape=(187, 1))
x = feature_extractor(input_layer, training=False)  # frozen convolutional base

#add transfer model here: transfer 6
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.3)(x)

output_layer = Dense(2, activation='softmax')(x)

transfer_model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
#lr exp dec: Learning rate with exponential decay
initial_learning_rate = 1e-3
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96)

#Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',        # what to monitor 
    patience=20,               # how many epochs with no improvement before stopping
    restore_best_weights=True, 
    min_delta=0.001            #only stop if improvement < 0.001
)

#Compile when lr exp decay
transfer_model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

# Define where and how to save the best model
checkpoint = ModelCheckpoint(
    filepath='../transfer/cnn8_sm_lrexpdec_earlystop_bs512_transfer6_model_lrexpdec_earlystop_bs512_epoch_{epoch:02d}_valloss_{val_loss:.4f}.keras',   # file path (can be .keras or .h5), change name when needed
    monitor='val_loss',            # metric to monitor
    mode='min',                    # minimize loss
    save_best_only=False,          # save model of every epoch
    verbose=1                      # print message when a model is saved
)

#training
history = transfer_model.fit(
    X_ptb_train_sm_cnn, #PTB data
    y_ptb_train_sm, #PTB data
    epochs=500,
    batch_size=512,
    validation_data=(X_ptb_val_cnn, y_ptb_val), #PTB data
    callbacks=[checkpoint, early_stop] 
)

In [ ]:
plot_training_history(history, save_dir="../transfer", prefix="cnn8_sm_lrexpdec_earlystop_bs512_transfer6_model_lrexpdec_earlystop_bs512") #change name when needed

In [ ]:
with open("transfer/cnn8_sm_lrexpdec_earlystop_bs512_transfer6_model_lrexpdec_earlystop_bs512.pkl", "wb") as f: #change name when needed
    pickle.dump(history.history, f)


best_model = load_model('../transfer/cnn8_sm_lrexpdec_earlystop_bs512_transfer6_model_lrexpdec_earlystop_bs512.keras') #change name when needed

test_pred = best_model.predict(X_ptb_test_cnn) #PTB data
y_test_class = y_ptb_test #PTB data
y_pred_class = np.argmax(test_pred, axis=1)


print(classification_report(y_test_class, y_pred_class, digits=4))

print(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions']))


#save results of metrics
with open("../transfer/cnn8_sm_lrexpdec_earlystop_bs512_transfer6_model_lrexpdec_earlystop_bs512.txt", "w") as file: # change name when needed
    file.write("\nModel: CNN8 MIT transfer6 PTB\n")
    file.write("\nData augmentation: MIT Smote, PTB Smote\n")
    file.write("\nConfusion Matrix on test set:\n")
    file.write(str(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions'])))
    file.write("\n\nClassification Report on test set:\n")
    file.write(classification_report(y_test_class, y_pred_class, digits=4))

In [ ]:
#retraining with last residual block unfrozen

In [ ]:
# Print all layers to see structure
print("Feature extractor layers:")
for i, layer in enumerate(feature_extractor.layers):
    print(f"{i}: {layer.name} - Trainable: {layer.trainable}")

In [ ]:
# Make feature extractor trainable
feature_extractor.trainable = True

# first: Freeze all layers
for layer in feature_extractor.layers:
    layer.trainable = False

# Unfreeze ONLY the last residual block (here: last 7 layers)
num_layers_last_block = 7  

for layer in feature_extractor.layers[-num_layers_last_block:]:
    layer.trainable = True
    print(f"Unfrozen: {layer.name}")

# Verification of trainable layers
print(f"\nTotal layers: {len(feature_extractor.layers)}")
print(f"Trainable layers: {sum([layer.trainable for layer in feature_extractor.layers])}")
print(f"Frozen layers: {sum([not layer.trainable for layer in feature_extractor.layers])}")

In [ ]:
# Build new classifier for 2 class problem

# Input same shape as original
input_layer = Input(shape=(187, 1))
x = feature_extractor(input_layer, training=False)  # frozen convolutional base

#add transfer model here: transfer 6
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.3)(x)

output_layer = Dense(2, activation='softmax')(x)

transfer_model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
#lr exp dec
# Learning rate with exponential decay
initial_learning_rate = 1e-3
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96)

#Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',        # what to monitor 
    patience=20,               # how many epochs with no improvement before stopping
    restore_best_weights=True, 
    min_delta=0.001            #only stop if improvement < 0.001
)

#Compile when lr exp decay
transfer_model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

# Define where and how to save the best model
checkpoint = ModelCheckpoint(
    filepath='../transfer/cnn8_sm_lrexpdec_earlystop_bs512_unfrozen_transfer6_model_lrexpdec_earlystop_bs512_epoch_{epoch:02d}_valloss_{val_loss:.4f}.keras',   # file path (can be .keras or .h5), change name when needed
    monitor='val_loss',            # metric to monitor
    mode='min',                    # minimize loss
    save_best_only=False,          # save model of every epoch
    verbose=1                      # print message when a model is saved
)

#training
history = transfer_model.fit(
    X_ptb_train_sm_cnn, #PTB data
    y_ptb_train_sm, #PTB data
    epochs=500,
    batch_size=512,
    validation_data=(X_ptb_val_cnn, y_ptb_val), #PTB data
    callbacks=[checkpoint, early_stop] 
)

In [ ]:
plot_training_history(history, save_dir="../transfer", prefix="cnn8_sm_lrexpdec_earlystop_bs512_unfrozen_transfer6_model_lrexpdec_earlystop_bs512") # change name when needed

In [ ]:
with open("../transfer/cnn8_sm_lrexpdec_earlystop_bs512_unfrozen_transfer6_model_lrexpdec_earlystop_bs512.pkl", "wb") as f: # change name when needed
    pickle.dump(history.history, f)


best_model = load_model('../transfer/cnn8_sm_lrexpdec_earlystop_bs512_unfrozen_transfer6_model_lrexpdec_earlystop_bs512.keras') # change name when needed

test_pred = best_model.predict(X_ptb_test_cnn) #PTB data
y_test_class = y_ptb_test #PTB data
y_pred_class = np.argmax(test_pred, axis=1)


print(classification_report(y_test_class, y_pred_class, digits=4))

print(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions']))


#save results of metrics
with open("../transfer/cnn8_sm_lrexpdec_earlystop_bs512_unfrozen_transfer6_model_lrexpdec_earlystop_bs512.txt", "w") as file: # change name when needed
    file.write("\nModel: CNN8 MIT, transfer6 PTB \n")
    file.write("\nData augmentation: MIT Smote, PTB Smote\n")
    file.write("\nConfusion Matrix on test set:\n")
    file.write(str(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions'])))
    file.write("\n\nClassification Report on test set:\n")
    file.write(classification_report(y_test_class, y_pred_class, digits=4))